In [89]:
import pandas as pd
import pickle
import numpy as np

import json

import time

import re

In [2]:
TREC_COVID_MERGED_FILE="trec_covid_merged_data.tsv"
TREC_COVID_DOCUMENTS_FILE="trec_covid_original_title_text_merged.tsv"

TREC_COVID_QRELS="trec_covid_qrels.tsv"

In [35]:
API_KEYS_FILE="../api_keys_20230324.json"

In [13]:
pd.set_option('display.max_colwidth', None)

## Set the random seed

In [15]:
RANDOM_SEED = 6

In [16]:
rng = np.random.default_rng(RANDOM_SEED)

## Explore TREC COVID documents with questions

In [ ]:
merged_df = pd.read_csv(TREC_COVID_MERGED_FILE, sep="\t")

In [4]:
merged_df

,query-id,corpus-id,score,query-text,corpus-title,corpus-text,query-metadata,corpus-metadata
0,1,005b2j4b,2,what is the origin of COVID-19,Monophyletic Relationship between Severe Acute...,Although primary genomic analysis has revealed...,"{'query': 'coronavirus origin', 'narrative': ""...",{'url': 'https://www.ncbi.nlm.nih.gov/pubmed/1...
1,16,005b2j4b,0,how long does coronavirus remain stable on su...,Monophyletic Relationship between Severe Acute...,Although primary genomic analysis has revealed...,{'query': 'how long does coronavirus survive o...,{'url': 'https://www.ncbi.nlm.nih.gov/pubmed/1...
2,32,005b2j4b,0,"Does SARS-CoV-2 have any subtypes, and if so w...",Monophyletic Relationship between Severe Acute...,Although primary genomic analysis has revealed...,"{'query': 'coronavirus subtypes', 'narrative':...",{'url': 'https://www.ncbi.nlm.nih.gov/pubmed/1...
3,37,005b2j4b,0,What is the result of phylogenetic analysis of...,Monophyletic Relationship between Severe Acute...,Although primary genomic analysis has revealed...,"{'query': 'SARS-CoV-2 phylogenetic analysis', ...",{'url': 'https://www.ncbi.nlm.nih.gov/pubmed/1...
4,1,00fmeepz,1,what is the origin of COVID-19,Comprehensive overview of COVID-19 based on cu...,"In December 2019, twenty-seven pneumonia patie...","{'query': 'coronavirus origin', 'narrative': ""...","{'url': '', 'pubmed_id': ''}"
...,...,...,...,...,...,...,...,...
66331,50,zn10rnrm,1,what is known about an mRNA vaccine for the SA...,Characterization of RNA in Saliva,Background: We have previously shown that huma...,"{'query': 'mRNA vaccine coronavirus', 'narrati...",{'url': 'https://www.ncbi.nlm.nih.gov/pmc/arti...
66332,50,zstmdt4n,0,what is known about an mRNA vaccine for the SA...,Coordinate induction of IFN-α and -γ by SARS-C...,Abstract Background: Severe acute respiratory ...,"{'query': 'mRNA vaccine coronavirus', 'narrati...",{'url': 'https://api.elsevier.com/content/arti...
66333,50,zth8ffy3,0,what is known about an mRNA vaccine for the SA...,Vasculopathy and Coagulopathy Associated with ...,The emergence of severe acute respiratory synd...,"{'query': 'mRNA vaccine coronavirus', 'narrati...","{'url': '', 'pubmed_id': ''}"
66334,50,zv4nbz9p,2,what is known about an mRNA vaccine for the SA...,"Emerging Technologies for Use in the Study, Di...",INTRODUCTION: The COVID-19 pandemic has caused...,"{'query': 'mRNA vaccine coronavirus', 'narrati...",{'url': 'https://doi.org/10.1007/s12195-020-00...


In [5]:
docs_with_questions = merged_df['corpus-id'].unique()

In [7]:
docs_with_questions.shape

(35480,)

In [304]:
which_docs = rng.choice(docs_with_questions, 100, replace=False)

In [305]:
which_docs

array(['hquc2v2c', '70hskj1o', 'z9rc0ubj', 'yao1vt02', 'qb8zxip9',
       'ntx2c04x', 'zu1ojh4e', '6jiltxrf', '5vda46uh', '34m7y2l1',
       '8rezgvz0', '1vba9l42', '7my48m8v', '24dtun4u', 'id2btkse',
       'x8vzuadu', 'se1rjm27', 'szpktvq4', 'hyk93ons', 'ey1sqch4',
       'igxdatq1', 'iq5yuwwv', '467nrx6d', '7lz06a7a', 'maclu8gh',
       '10pgl1v3', '7k36owrf', 'vqpmipvk', 'asw56fla', 'ac38yvcn',
       'xqmvix1o', 'swwz4kzd', 'gryp0khc', 'k0l4810i', 'jewffjwj',
       'ekmuz4ug', 'dvgqouk2', 'dkzqfs7m', 'r0y96ou2', '8w2avwo2',
       'nqrtk06s', 'hj0t4wjj', 'h4s25x7x', '0iburamm', 'tcc7e6ls',
       '4fu922cp', '3xw4qjoy', '77tww0a8', 't4os33em', 'szr37ovl',
       '3d8up3zo', 'ubhntliy', '0sbaxwuf', 'd4yidznm', 'hr3vkjh3',
       'jyfcpsu6', 'kimyvw4w', 'fdxmh9uo', '41h59d0g', 'mlwrfkm4',
       'tmpidjrp', '7u6tdenk', 'kfx0qvkp', 'fac4ic30', 'krvei97r',
       'fy9g02af', '08zf7161', 'nbcjv47q', 'c7khkkcj', 'pn6379q6',
       'mp6swq36', 'b8a78ym6', '3zy5dgxz', 'gjkdm90a', '1ycins

In [306]:
merged_df[merged_df['corpus-id'] == which_docs[0]]

,query-id,corpus-id,score,query-text,corpus-title,corpus-text,query-metadata,corpus-metadata
57427,35,hquc2v2c,0,What new public datasets are available related to COVID-19?,A Comprehensive Analysis of COVID-19 Outbreak situation in India,"The outbreak of COVID-19 in different parts of the world is a major concern for all the administrative units of respective countries. India is also facing this very tough task for controlling the virus outbreak and has managed its growth rate through some strict measures. This study presents the current situation of coronavirus spread in India along with the impact of various measures taken for it. With the help of data sources (till 7th-8th April 2020) from various state units of India and Ministry of Health and Family Welfare, Government of India, this study presents various trends and patterns. This study answers six different research ques-tions in a comprehensive manner. It has been reported that growth rate of infected cases has been controlled with the help of National Lockdown, however some uncontrolled mass level events had negatively impacted the infected cases. With the help of exponential and polyno-mial regression modelling, the predictions of up to 75000 cases have been done by the end of April 2020. It has also been seen that there are some prominent clusters and patient nodes in the network of patients which are the major influencers for COVID-19 spread. Also, death rate case predictions have been done through multi-class classification models with an accuracy of 75%. At the end, strategies for continuation for lockdown has been discussed and presented. It appears that only essential services should be open for the citizens of India and the national lockdown should be carried on for next 2-4 weeks. This study will be useful for the Government of India and various states of India, Administrative Units of India, Frontline health workforce of India, researchers and scientists. This study will also be favorable for the administrative units of other countries to consider various aspects related to the control of COVID-19 outspread in their respective regions.","{'query': 'coronavirus public datasets', 'narrative': 'Seeking articles that specifically release new data related to SARS-CoV-2 or COVID-19, including genomic data, patient data, public health data, etc. Articles that reference previously existing datasets are not relevant.'}","{'url': 'https://doi.org/10.1101/2020.04.08.20058347', 'pubmed_id': ''}"


In [307]:
doc_text = merged_df[merged_df['corpus-id'] == which_docs[0]]['corpus-text'].to_numpy()[0]

In [308]:
doc_text

'The outbreak of COVID-19 in different parts of the world is a major concern for all the administrative units of respective countries. India is also facing this very tough task for controlling the virus outbreak and has managed its growth rate through some strict measures. This study presents the current situation of coronavirus spread in India along with the impact of various measures taken for it. With the help of data sources (till 7th-8th April 2020) from various state units of India and Ministry of Health and Family Welfare, Government of India, this study presents various trends and patterns. This study answers six different research ques-tions in a comprehensive manner. It has been reported that growth rate of infected cases has been controlled with the help of National Lockdown, however some uncontrolled mass level events had negatively impacted the infected cases. With the help of exponential and polyno-mial regression modelling, the predictions of up to 75000 cases have been 

## Generate some questions using OpenAI

In [40]:
!python3.8 -m pip install openai -q

You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.


In [75]:
import openai

In [309]:
OPENAI_API_QUERY_PARAMS={"model": None,
                         "prompt": None,
                         "temperature": 0.7,
                         "max_tokens": 512,
                         "top_p": 1.0,
                         "frequency_penalty": 0,
                         "presence_penalty": 0}

In [310]:
ZERO_SHOT_TEMPLATE="Considering the text in quotes below:\n\n\"{}\"\n\nWhich questions can you drawn about non-trivial conclusions from the text?"

In [311]:
OPENAI_RESPONSE_REGEX="[\n\r]*[0-9\.]+\s+(.+)[\n\r]?"

In [312]:
with open(API_KEYS_FILE) as inputFile:
    api_keys = json.load(inputFile)

In [313]:
openai.api_key = api_keys['OPENAI_API_KEY']

In [314]:
request_params = OPENAI_API_QUERY_PARAMS

In [315]:
request_params['prompt'] = ZERO_SHOT_TEMPLATE.format(doc_text)
request_params['model'] = 'text-davinci-003'

In [316]:
request_params['prompt']

'Considering the text in quotes below:\n\n"The outbreak of COVID-19 in different parts of the world is a major concern for all the administrative units of respective countries. India is also facing this very tough task for controlling the virus outbreak and has managed its growth rate through some strict measures. This study presents the current situation of coronavirus spread in India along with the impact of various measures taken for it. With the help of data sources (till 7th-8th April 2020) from various state units of India and Ministry of Health and Family Welfare, Government of India, this study presents various trends and patterns. This study answers six different research ques-tions in a comprehensive manner. It has been reported that growth rate of infected cases has been controlled with the help of National Lockdown, however some uncontrolled mass level events had negatively impacted the infected cases. With the help of exponential and polyno-mial regression modelling, the p

In [317]:
request_start_time = time.time()

response = openai.Completion.create(**request_params)

elapsed_time = time.time() - request_start_time

In [318]:
print(elapsed_time)

4.19231104850769


In [319]:
print(response['choices'][0]['text'])



1. What measures has India taken to control the spread of COVID-19?
2. What trends and patterns are seen in the spread of COVID-19 in India?
3. How accurate are the predictions of COVID-19 cases and death rate in India?
4. What strategies should India consider for continuation of the national lockdown?
5. How can this study be useful for the Government of India and other countries?
6. Are there any major influencers for COVID-19 spread in India?


In [320]:
m = re.finditer(OPENAI_RESPONSE_REGEX, response['choices'][0]['text'])

In [321]:
questions = [match.group(1) for match in m]

In [322]:
len(questions)

6

In [323]:
questions

['What measures has India taken to control the spread of COVID-19?',
 'What trends and patterns are seen in the spread of COVID-19 in India?',
 'How accurate are the predictions of COVID-19 cases and death rate in India?',
 'What strategies should India consider for continuation of the national lockdown?',
 'How can this study be useful for the Government of India and other countries?',
 'Are there any major influencers for COVID-19 spread in India?']

In [324]:
new_dataset = []

for question in questions:
    new_dataset.append({'query': question,
                        'positive_doc_id': which_doc,
                        'negative_doc_ids': []})

In [325]:
pd.DataFrame(new_dataset)

,query,positive_doc_id,negative_doc_ids
0,What measures has India taken to control the spread of COVID-19?,[i7scw9mu],[]
1,What trends and patterns are seen in the spread of COVID-19 in India?,[i7scw9mu],[]
2,How accurate are the predictions of COVID-19 cases and death rate in India?,[i7scw9mu],[]
3,What strategies should India consider for continuation of the national lockdown?,[i7scw9mu],[]
4,How can this study be useful for the Government of India and other countries?,[i7scw9mu],[]
5,Are there any major influencers for COVID-19 spread in India?,[i7scw9mu],[]
